In [1]:
%pip install llama-index


  Obtaining dependency information for llama-index from https://files.pythonhosted.org/packages/eb/52/0584431abd6cbbc85def8ac3b38083a867093764d5f831889b4b1c44b44e/llama_index-0.10.9-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index-agent-openai<0.2.0,>=0.1.0 from https://files.pythonhosted.org/packages/18/0e/6424359193d4a4aabca5ae62e4ffc11b3c8e09a989a58c50d6b4ada83124/llama_index_agent_openai-0.1.3-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index-core<0.11.0,>=0.10.8.post1 from https://files.pythonhosted.org/packages/c5/5f/9f90f60a22925e69fbea0e80ae785ebce63022f104d34ef41818edf045be/llama_index_core-0.10.8.post1-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index-embeddings-openai<0.2.0,>=0.1.0 from https://files.pythonhosted.org/packages/46/b5/51e2801850216e6ac22aa2822c1d3f17d6ca5f4224f4cf60bece92863ca3/llama_index_embeddings_openai-0.1.4-py3-none-any.whl.metadata
  Obtaining dependency information for ll


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\arnav\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
%pip install llama-index-llms-openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\arnav\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [90]:
import os

os.environ["OPENAI_API_KEY"] = "sk-"

import nest_asyncio

nest_asyncio.apply()

In [91]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [92]:
# LLM (gpt-3.5)
gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")

# LLM (gpt-4)
gpt4 = OpenAI(temperature=0, model="gpt-4")

In [93]:
%mkdir -p 'data/paul_graham/'
%wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

zsh:1: command not found: wget


In [94]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [95]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

In [96]:
index = VectorStoreIndex.from_documents(documents)

**********
Trace: index_construction
    |_CBEventType.EMBEDDING ->  0.373351 seconds
**********


In [97]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

# gpt-4
step_decompose_transform = StepDecomposeQueryTransform(llm=gpt4, verbose=True)

# gpt-3
step_decompose_transform_gpt3 = StepDecomposeQueryTransform(
    llm=gpt35, verbose=True
)

In [98]:
index_summary = "Used to answer questions about the author"

In [108]:
# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine import MultiStepQueryEngine

query_engine = index.as_query_engine(llm=gpt4)
query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform,
    index_summary=index_summary,
)
response_gpt4 = query_engine.query(
    "What is the article about?", 
)

> Current query: What is the article about?
> New query: Who is the author of the article?
> Current query: What is the article about?
> New query: What is the topic of the articles written by Paul Graham?
> Current query: What is the article about?
> New query: What is the specific topic of this article by Paul Graham?
**********
Trace: query
    |_CBEventType.QUERY ->  8.20043 seconds
      |_CBEventType.TEMPLATING ->  4.8e-05 seconds
      |_CBEventType.LLM ->  0.696667 seconds
      |_CBEventType.QUERY ->  0.86932 seconds
        |_CBEventType.RETRIEVE ->  0.185336 seconds
          |_CBEventType.EMBEDDING ->  0.181497 seconds
        |_CBEventType.SYNTHESIZE ->  0.683744 seconds
          |_CBEventType.TEMPLATING ->  3.9e-05 seconds
          |_CBEventType.LLM ->  0.670703 seconds
      |_CBEventType.TEMPLATING ->  4.9e-05 seconds
      |_CBEventType.LLM ->  0.676338 seconds
      |_CBEventType.QUERY ->  1.145946 seconds
        |_CBEventType.RETRIEVE ->  0.106204 seconds
        

In [100]:
# extract the most new query

In [101]:
# build index and query engine
vector_query_engine = VectorStoreIndex.from_documents(
    documents,
    use_async=True,
).as_query_engine()

**********
Trace: index_construction
    |_CBEventType.EMBEDDING ->  0.688091 seconds
**********


In [102]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="documents",
            description="Paul Graham essay on What I Worked On",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

In [109]:
response = query_engine.query(
    # instead of this, use extracted query 
    #To Do: reference newest query here 
    "What is the specific topic of this article by Paul Graham?"
)

> Current query: What is the specific topic of this article by Paul Graham?
> New query: Who is Paul Graham?
> Current query: What is the specific topic of this article by Paul Graham?
> New query: What is the content of the article written by Paul Graham?
> Current query: What is the specific topic of this article by Paul Graham?
> New query: What is the journey of Paul Graham as a writer, programmer, and entrepreneur as discussed in the article?
**********
Trace: query
    |_CBEventType.QUERY ->  31.91208 seconds
      |_CBEventType.TEMPLATING ->  3.5e-05 seconds
      |_CBEventType.LLM ->  0.558577 seconds
      |_CBEventType.QUERY ->  2.653315 seconds
        |_CBEventType.RETRIEVE ->  0.116409 seconds
          |_CBEventType.EMBEDDING ->  0.110459 seconds
        |_CBEventType.SYNTHESIZE ->  2.536632 seconds
          |_CBEventType.TEMPLATING ->  4.3e-05 seconds
          |_CBEventType.LLM ->  2.521596 seconds
      |_CBEventType.TEMPLATING ->  1.7e-05 seconds
      |_CBEventType.

In [110]:
# iterate through sub_question items captured in SUB_QUESTION event
from llama_index.core.callbacks import CBEventType, EventPayload

for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
    print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    print("Answer: " + qa_pair.answer.strip())
    print("====================================")

Sub Question 0: What is the main topic of Paul Graham's article?
Answer: The main topic of Paul Graham's article is his journey and experiences in the tech industry, including his work on developing a new Lisp, the impact of publishing essays online, his interactions with startup founders, and the founding of Y Combinator.
Sub Question 1: What are the supporting ideas in Paul Graham's article?
Answer: The supporting ideas in Paul Graham's article include the importance of being drawn to work despite its lack of prestige, the danger of having impure motives, the value of working on projects that may not be prestigious but have real potential, the significance of avoiding the desire to impress others, and the benefits of focusing on genuine interests rather than seeking external validation.
Sub Question 2: What is the frame of Paul Graham's article?
Answer: The frame of Paul Graham's article is his personal journey and experiences in exploring different projects and ideas, including his 

In [111]:
print(response)

The specific topic of this article by Paul Graham is his journey as a writer, programmer, and entrepreneur, reflecting on his experiences with online publishing, working on projects that may not be prestigious but are personally meaningful, his involvement in Y Combinator, and his various endeavors in the tech and startup world.
